In [26]:
import requests
from datetime import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

# import sys
import requests
url = 'https://www.wikidata.org/w/api.php'

def search_entities(name, url):
    """
    Get the entity id, using a name, by calling the Wiki API. 
    The output is a list of dict of dictionaries corresponding to the number of
    occurances of the given name 
    """
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': name
    }
    response = requests.get(url, params=params)
    return response.json()['search']



In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sys

# endpoint_url = "https://query.wikidata.org/sparql"

# query = """
# SELECT ?item ?occupationLabel ?image ?genderLabel ?bdayLabel ?citizenshipLabel
# WHERE 
# {
#   ?item wdt:P106 ?occupation .
#   ?item wdt:P21 ?gender .
#   ?item wdt:P18 ?image .
#   ?item wdt:P569 ?bday .
#   ?item wdt:P27 ?citizenship
#   FILTER(?item = wd:Q37079)
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
# }
# """


def get_results(endpoint_url, query):
    """
    Get's the information from endpoint (wikidata), by submitting a SPAQL query,
    returning s corresponding JSON file
    """
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


# results = get_results(endpoint_url, query)

# for result in results["results"]["bindings"]:
#     print(result)

In [4]:
import pandas as pd
# Get all the names and entity_id
data = pd.read_csv("../raw_data/list_act.csv") #read current names from the list_act csv

In [5]:
#new dictionary to ho
new_file_dict = {
    "name":[],
    "wiki_id":[]
}
for name in data["name"]:
    res = search_entities(name,url=url)[0]["id"]
    new_file_dict["name"].append(name)
    new_file_dict["wiki_id"].append(res)
   

In [6]:
df_act = pd.DataFrame(new_file_dict)


In [7]:
df_act.to_csv("../raw_data/celebrity_list.csv")

In [8]:
df_act = pd.read_csv("../raw_data/celebrity_list.csv")
import sys
sys.path.insert(0, '../')
import pandas as pd


In [9]:
# Create a new df with all the names taken from the celebrity.csv having wiki_id and that are death
from logic import WikiDataQueryResults as wdr


new_df = pd.DataFrame(columns=['itemLabel','item', 'image', 'occupationLabel','genderLabel', 'bdayLabel','citizenshipLabel','dodLabel'])

for wiki_id in df_act["wiki_id"]:

    query = f"""
    SELECT ?itemLabel ?item ?occupationLabel ?image ?genderLabel ?bdayLabel ?citizenshipLabel ?dodLabel
    WHERE 
    {{
    ?item wdt:P106 ?occupation .
    ?item wdt:P21 ?gender .
    ?item wdt:P18 ?image .
    ?item wdt:P569 ?bday .
    ?item wdt:P27 ?citizenship .
    ?item wdt:P570 ?dod .
    FILTER(?item = wd:{wiki_id})
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """

    data_extracter = wdr(query)
    df = data_extracter.load_as_dataframe()
    new_df = pd.concat([new_df,df[0:1]])
new_df.to_csv("../raw_data/metafile.csv") #write to csv file

In [10]:
new_df.to_csv("../raw_data/metafile.csv")


In [11]:
# Create a new df with all the names taken from the celebrity.csv having wiki_id all
from logic import WikiDataQueryResults as wdr


new_df = pd.DataFrame(columns=['itemLabel','item', 'image', 'occupationLabel','genderLabel', 'bdayLabel','citizenshipLabel'])

for wiki_id in df_act["wiki_id"]:

    query = f"""
    SELECT ?itemLabel ?item ?occupationLabel ?image ?genderLabel ?bdayLabel ?citizenshipLabel
    WHERE 
    {{
    ?item wdt:P106 ?occupation .
    ?item wdt:P21 ?gender .
    ?item wdt:P18 ?image .
    ?item wdt:P569 ?bday .
    ?item wdt:P27 ?citizenship .
    FILTER(?item = wd:{wiki_id})
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """

    data_extracter = wdr(query)
    df = data_extracter.load_as_dataframe()
    new_df = pd.concat([new_df,df])
new_df.to_csv("../raw_data/metafile1.csv") #write to csv file

In [12]:
df_test = pd.read_csv("../raw_data/metafile1.csv")


In [13]:
df_test.columns

Index(['Unnamed: 0', 'itemLabel', 'item', 'image', 'occupationLabel',
       'genderLabel', 'bdayLabel', 'citizenshipLabel'],
      dtype='object')

In [14]:
#Aggregate all rows with the first citizenship, genderLabe and bdayLabel
df_test = df_test.groupby(['occupationLabel','itemLabel', ],as_index=False).agg({'itemLabel' : 'first', 'genderLabel':'first', 'bdayLabel':'first', 'citizenshipLabel':'first'})

In [15]:
df_test[df_test["itemLabel"]=="Cate Blanchett"]

,occupationLabel,itemLabel,genderLabel,bdayLabel,citizenshipLabel
29,actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
369,film actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
545,film director,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
633,film producer,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
734,international forum participant,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
990,stage actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
1263,television producer,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
1314,voice actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America


In [16]:
df_test = df_test.groupby('itemLabel', as_index=False).agg({'itemLabel' : 'first', 'occupationLabel' : ', '.join,'genderLabel':'first', 'bdayLabel':'first', 'citizenshipLabel':'first'})

In [27]:
df_test.to_csv("../raw_data/metafile3.csv")

In [28]:
file_path = '../raw_data/metafile3.csv'
metafile_df = pd.read_csv(file_path)
metafile_df = metafile_df.iloc[:,1:]

In [29]:
metafile2_df = pd.read_csv('../raw_data/metafile.csv')
metafile2_df = metafile2_df.iloc[:,1:].drop(columns=['item','image'])
metafile2_df

,itemLabel,occupationLabel,genderLabel,bdayLabel,citizenshipLabel,dodLabel
0,Marlene Dietrich,actor,female,1901-12-27T00:00:00Z,United States of America,1992-05-06T00:00:00Z
1,Marilyn Monroe,actor,female,1926-06-01T00:00:00Z,United States of America,1962-08-04T00:00:00Z
2,Ingrid Bergman,actor,female,1915-08-29T00:00:00Z,Sweden,1982-08-29T00:00:00Z
3,Bette Davis,actor,female,1908-04-05T00:00:00Z,United States of America,1989-10-06T00:00:00Z
4,Bibi Andersson,stage actor,female,1935-11-11T00:00:00Z,Sweden,2019-04-14T00:00:00Z
5,Vivien Leigh,actor,female,1913-11-05T00:00:00Z,United Kingdom,1967-07-08T00:00:00Z
6,Janet Leigh,actor,female,1927-07-06T00:00:00Z,United States of America,2004-10-03T00:00:00Z
7,Virginia Cherrill,actor,female,1908-04-12T00:00:00Z,United States of America,1996-11-14T00:00:00Z
8,Grace Kelly,actor,female,1929-11-12T00:00:00Z,United States of America,1982-09-14T00:00:00Z
9,Gloria Swanson,actor,female,1899-03-27T00:00:00Z,United States of America,1983-04-04T00:00:00Z


In [30]:
hint_df = pd.merge(left=metafile_df, right=metafile2_df[['itemLabel','dodLabel']], on='itemLabel', how='left')
hint_df

,itemLabel,occupationLabel,genderLabel,bdayLabel,citizenshipLabel,dodLabel
0,Adèle Exarchopoulos,"actor, film actor, model",female,1993-11-22T00:00:00Z,France,NaN
1,Al Pacino,"film actor, film producer, stage actor, televi...",male,1940-04-25T00:00:00Z,United States of America,NaN
2,Alan Arkin,"character actor, children's writer, film actor...",male,1934-03-26T00:00:00Z,United States of America,NaN
3,Alan Rickman,"actor, character actor, director, film actor, ...",male,1946-02-21T00:00:00Z,United Kingdom,2016-01-14T00:00:00Z
4,Albert Finney,"actor, film actor, film director, film produce...",male,1936-05-09T00:00:00Z,United Kingdom,2019-02-07T00:00:00Z
...,...,...,...,...,...,...
202,Virginia Cherrill,"actor, film actor",female,1908-04-12T00:00:00Z,United States of America,1996-11-14T00:00:00Z
203,Vivien Leigh,"actor, film actor, stage actor",female,1913-11-05T00:00:00Z,United Kingdom,1967-07-08T00:00:00Z
204,Will Smith,"actor, film producer, singer",male,1968-09-25T00:00:00Z,United States of America,NaN
205,William Holden,"actor, film actor, television actor",male,1918-04-17T00:00:00Z,United States of America,1981-11-12T00:00:00Z


In [31]:
hint_df.columns = ['Celebrity', 'Occupation', 'Gender', 'Birthday', 'Citizenship', 'Death_date']
hint_df

,Celebrity,Occupation,Gender,Birthday,Citizenship,Death_date
0,Adèle Exarchopoulos,"actor, film actor, model",female,1993-11-22T00:00:00Z,France,NaN
1,Al Pacino,"film actor, film producer, stage actor, televi...",male,1940-04-25T00:00:00Z,United States of America,NaN
2,Alan Arkin,"character actor, children's writer, film actor...",male,1934-03-26T00:00:00Z,United States of America,NaN
3,Alan Rickman,"actor, character actor, director, film actor, ...",male,1946-02-21T00:00:00Z,United Kingdom,2016-01-14T00:00:00Z
4,Albert Finney,"actor, film actor, film director, film produce...",male,1936-05-09T00:00:00Z,United Kingdom,2019-02-07T00:00:00Z
...,...,...,...,...,...,...
202,Virginia Cherrill,"actor, film actor",female,1908-04-12T00:00:00Z,United States of America,1996-11-14T00:00:00Z
203,Vivien Leigh,"actor, film actor, stage actor",female,1913-11-05T00:00:00Z,United Kingdom,1967-07-08T00:00:00Z
204,Will Smith,"actor, film producer, singer",male,1968-09-25T00:00:00Z,United States of America,NaN
205,William Holden,"actor, film actor, television actor",male,1918-04-17T00:00:00Z,United States of America,1981-11-12T00:00:00Z


In [32]:
hint_df.Citizenship = hint_df.Citizenship.apply(lambda x: 'USA' if x == 'United States of America' else x) \
                                         .apply(lambda x: 'Ireland' if x == 'Republic of Ireland' else x) \
                                         .apply(lambda x: 'The UK' if x == 'United Kingdom' else x)
hint_df.Citizenship.value_counts()

USA            141
The UK          30
France           8
Canada           7
Australia        6
Ireland          5
Sweden           3
Spain            2
Austria          1
Italy            1
Germany          1
Norway           1
New Zealand      1
Name: Citizenship, dtype: int64

In [33]:
def calculate_age(row):
    birth_date = datetime.strptime(row['Birthday'][:10], r'%Y-%m-%d').date()
    
    if pd.isnull(row['Death_date']):
        present_date = datetime.now().date()
        age = present_date.year - birth_date.year - ((present_date.month, present_date.day) < (birth_date.month, birth_date.day))
        return f'Born in {birth_date.year}. Age: {age}'
    else:
        death_date = datetime.strptime(row['Death_date'][:10], r'%Y-%m-%d').date()
        age = death_date.year - birth_date.year - ((death_date.month, death_date.day) < (birth_date.month, birth_date.day))
        return f'{birth_date.year}-{death_date.year}. Age: {age}'

In [34]:
hint_df['Age'] = hint_df.apply(calculate_age, axis=1)
hint_df = hint_df.drop(columns=['Birthday', 'Death_date'])

In [35]:
hint_df['Occupation'] = hint_df['Occupation'].apply(lambda x: ", ".join(x.split(', ')[:3]) if len(x.split(', ')) > 3 else x)

In [36]:
hint_df.to_csv("../raw_data/metafile_complete.csv")

In [37]:
#Import Images from Wikidata - optional

import os
import unicodedata

user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1]) #define agent
headers = {'User-Agent': user_agent}

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# Iterate over all celebrities in df and get image and save to output_wiki
for idx, row in new_df.iterrows():
    img_url = new_df["image"].iloc[idx]
    # person_name = new_df["itemLabel"].iloc[idx]
    person_name = "_".join(strip_accents(new_df["itemLabel"].iloc[idx].lower()).split())+".jpg"
    target_path = os.path.join(os.getcwd(),"raw_data","output_wiki",person_name)
    target_image = target_path
    response = requests.get(img_url,stream=True,headers=headers)

    if response.status_code:
        fp = open(target_image, 'wb')
        fp.write(response.content)
        fp.close()


FileNotFoundError: [Errno 2] No such file or directory: '/home/jal/code/stan-kondratev/celebrities-identification-quest/notebooks/raw_data/output_wiki/cate_blanchett.jpg'